In [23]:
!pip install git+https://github.com/huggingface/transformers -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [1]:
%cd drive/MyDrive/data

/content/drive/MyDrive/data


In [2]:
import pandas as pd

train_df = pd.read_csv("train.csv")
train_df.head()

,ID,Utterance,Speaker,Dialogue_ID,Target
0,TRAIN_0000,also I was the point person on my company’s tr...,Chandler,0,neutral
1,TRAIN_0001,You must’ve had your hands full.,The Interviewer,0,neutral
2,TRAIN_0002,That I did. That I did.,Chandler,0,neutral
3,TRAIN_0003,So let’s talk a little bit about your duties.,The Interviewer,0,neutral
4,TRAIN_0004,My duties? All right.,Chandler,0,surprise


In [3]:
train_df.groupby(by=['Target']).count()

,ID,Utterance,Speaker,Dialogue_ID
Target,,,,
anger,1109,1109,1109,1109
disgust,271,271,271,271
fear,268,268,268,268
joy,1743,1743,1743,1743
neutral,4710,4710,4710,4710
sadness,683,683,683,683
surprise,1205,1205,1205,1205


In [4]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

label_encoder = LabelEncoder()
label_encoder.fit(train_df['Target'])
num_labels = len(label_encoder.classes_)

train_df['encoded_target'] = np.asarray(label_encoder.transform(train_df['Target']), dtype=np.int32)

In [5]:
train_df.head()

,ID,Utterance,Speaker,Dialogue_ID,Target,encoded_target
0,TRAIN_0000,also I was the point person on my company’s tr...,Chandler,0,neutral,4
1,TRAIN_0001,You must’ve had your hands full.,The Interviewer,0,neutral,4
2,TRAIN_0002,That I did. That I did.,Chandler,0,neutral,4
3,TRAIN_0003,So let’s talk a little bit about your duties.,The Interviewer,0,neutral,4
4,TRAIN_0004,My duties? All right.,Chandler,0,surprise,6


In [6]:
train_texts = train_df['Utterance'].to_list()
train_labels = train_df['encoded_target'].to_list()

In [7]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, 
                                                                    test_size=0.2, random_state=42)

In [24]:
from transformers import AutoTokenizer

model_name = "tae898/emoberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case = True)

In [33]:
train_tokenizer = tokenizer(train_texts, truncation = True, padding = True, 
                            max_length = 128, return_tensors = "pt")
val_tokenizer = tokenizer(val_texts, truncation = True, padding = True,
                          max_length = 128, return_tensors = "pt")

### tf

In [29]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_tokenizer), train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_tokenizer), val_labels))

In [14]:
!pip install tensorflow-addons -q

     |████████████████████████████████| 1.1 MB 8.1 MB/s 


In [30]:
from transformers import AutoModelForSequenceClassification
import tensorflow_addons.metrics

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels = num_labels)

In [31]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, 
              metrics=['accuracy',tensorflow_addons.metrics.F1Score(num_classes=num_labels, average = 'macro')])

AttributeError: ignored

### pt

In [34]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_labels)

In [ ]:
outputs = model(**train_tokenizer)

In [ ]:
import 